In [1]:
import pandas as pd
import numpy as np

import os

import utils

import warnings
warnings.filterwarnings('ignore')

In [2]:
data_dir = '../data/'
filenames = os.listdir(data_dir)
for i, ele in enumerate(filenames):
    print(i, ele)

0 olist_customers_dataset.csv
1 olist_geolocation_dataset.csv
2 olist_orders_dataset.csv
3 olist_order_items_dataset.csv
4 olist_order_payments_dataset.csv
5 olist_order_reviews_dataset.csv
6 olist_products_dataset.csv
7 olist_sellers_dataset.csv
8 processed
9 product_category_name_translation.csv


## Orders data

In [3]:
df_orders = pd.read_csv(data_dir+filenames[2], parse_dates=['order_purchase_timestamp',
                                                            'order_approved_at', 
                                                            'order_delivered_carrier_date',
                                                            'order_delivered_customer_date',
                                                            'order_estimated_delivery_date'])

df_orders['order_status'] = df_orders['order_status'].astype('category').cat.codes.values
df_orders = utils.reduce_mem_usage(df_orders)

Mem. usage decreased to  5.41 Mb (0.0% reduction)


In [4]:
utils.to_pickles(df_orders, '../data/processed/orders')

3it [00:00,  6.46it/s]


## Payments data

In [5]:
df_payments = pd.read_csv(data_dir+filenames[4])

df_payments['payment_type'] = df_payments['payment_type'].astype('category').cat.codes.values

df_payments = utils.reduce_mem_usage(df_payments)

Mem. usage decreased to  1.29 Mb (60.6% reduction)


In [6]:
utils.to_pickles(df_payments, '../data/processed/payments')

3it [00:00, 30.77it/s]


## Customers data

In [7]:
df_customers = pd.read_csv(data_dir+filenames[0])

df_customers = df_customers.merge(df_orders[['order_id', 'customer_id']], on='customer_id')

t = df_customers['customer_state'].astype('category')  ##store the cat codes, use it for seller info
cust_state = dict(zip(t, t.cat.codes))

t = df_customers['customer_city'].astype('category')  ##store the cat codes, use it for seller info
cust_city = dict(zip(t, t.cat.codes))

df_customers['customer_state'] = df_customers['customer_state'].astype('category').cat.codes.values
df_customers['customer_city'] = df_customers['customer_city'].astype('category').cat.codes.values


df_customers = utils.reduce_mem_usage(df_customers)

Mem. usage decreased to  3.70 Mb (9.3% reduction)


In [8]:
utils.to_pickles(df_customers, '../data/processed/customers')

3it [00:00,  6.42it/s]


## Products data

In [9]:
df_products = pd.read_csv(data_dir+filenames[6])

df_products['product_category_name'] = df_products['product_category_name'].astype('category').cat.codes

df_products = utils.reduce_mem_usage(df_products)

Mem. usage decreased to  0.72 Mb (64.6% reduction)


In [10]:
utils.to_pickles(df_products, '../data/processed/products')

3it [00:00, 37.12it/s]


## sellers data

In [11]:
df_sellers = pd.read_csv(data_dir+filenames[7])

df_sellers['seller_city'] = df_sellers['seller_city'].astype('category').map(cust_city)
df_sellers['seller_city'].fillna(-1, inplace=True)    ##reuse cat codes from customer info

df_sellers['seller_state'] = df_sellers['seller_state'].astype('category').map(cust_state)
df_sellers['seller_state'].fillna(-1, inplace=True)   ##reuse cat codes from customer info

df_sellers = utils.reduce_mem_usage(df_sellers)

Mem. usage decreased to  0.04 Mb (53.1% reduction)


In [12]:
utils.to_pickles(df_sellers, '../data/processed/sellers')

3it [00:00, 51.22it/s]
